## D3 JavaScript visualisation in a Python Jupyter notebook

### JavaScript in a Python Jupyter notebook cell

In [ ]:
%%javascript
console.log("I'm writing JavaScript");

Check Console in Developer Tools (instructions below), where you can see the result of JavaScript print statements. Ignore other messages already there relating to Jupyter functionality. "I'm writing JavaScript" should be printed at the bottom.

**To access Console in Developer Tools:** 

From Chrome browser: View > Developer > JavaScript Console

From Firefox browser: Tools > Web Developer > Web Console

From Safari, first turn on the “Develop menu”: Safari > Preferences > Advanced. Select 'Show Develop menu in menu bar' and close Preferences. Then go Develop > Show JavaScript Console

In [ ]:
from IPython.display import Javascript 
Javascript("""
console.log("I'm also writing JavaScript here");
""")
# check Console in Developer Tools again

### Importing D3

In [ ]:
%%javascript
require.config({
    paths: { 
        d3: 'https://d3js.org/d3.v5.min'
    }
});

In [ ]:
%%javascript
require(['d3'], function(d3) {   
    console.log(d3);
});
// Check Console in Developer tools again - you'll see an Object that is D3. 
// Depending on your browser it may just say 'Object' and you can use the small arrow 
// to unfurl it to see the D3 library contents,  
// or it looks like '{event: null, format: ƒ, formatPrefix: ƒ, timeFormat: ƒ, timeParse: ƒ, …}'

### Accessing the output cell

Running JavaScript, you don’t automatically get anything appearing in the output cell. You can use 'element' (jQuery powered wrapper) to access the current output cell.

In [ ]:
%%javascript
element.text('hello world');

'element.get(0)' is the [DOM](https://www.w3schools.com/js/js_htmldom.asp) node for the output cell that can be handed to 'd3.select()'. Wrap this in a [closure](https://www.w3schools.com/js/js_function_closures.asp) to make sure you're accessing the correct output cell, as 'element' is a global variable and is overwritten when new cells are run

In [ ]:
%%javascript
(function(element) {
    require(['d3'], function(d3) {   
        d3.select(element.get(0)).append('text').text('hello world');
    })
})(element);

In [ ]:
%%javascript
(function(element) {
    require(['d3'], function(d3) {   
        d3.select(element.get(0)).append('svg')
            .append('rect')
            .attr('width', 50)
            .attr('height', 50)
            .attr('fill', 'black')
            .on('mouseover', function(){d3.select(this).attr('fill', 'red')})
            .on('mouseout', function(){d3.select(this).attr('fill', 'black')});
    })
})(element);

// This rectangle is interactive - mouseover to see it turn red 

### Getting your data from Python into JavaScript

In [ ]:
import json

In [ ]:
data = [50, 100]

Pass data into a JavaScript command

In [ ]:
Javascript("""
element.text(%s);
""" % json.dumps(data))

# Triple quotes allows a multi-line string in Python
# I'm using this Python string formatting method: 
# https://matthew-brett.github.io/teaching/string_formatting.html#option-3-old-school-formatting

In [ ]:
Javascript("""
(function(element) {
    require(['d3'], function(d3) {   
        var data = %s;
        d3.select(element.get(0)).append('svg')
        .append('rect')
        .attr('width', data[0])
        .attr('height', data[1])
        .attr('fill', 'black')
    })
})(element);
""" % json.dumps(data))

# This rectangle is 50 x 100, from our data

In [ ]:
# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
data_python = [['A', 10], ['B', 15], ['C', 14]] 
  
# Create the pandas DataFrame 
df = pd.DataFrame(data_python, columns = ['Name', 'Count']) 
  
# print dataframe. 
df

In [ ]:
#Save df as json
df.to_json(r'df.json', orient="records")

In [ ]:
%%javascript
(function(element) {
    require(['d3'], function(d3) {   
        // access df.json created by the cell above
        d3.json("df.json").then(function(json){
            var df_data = json;            
            var svg = d3.select(element.get(0)).append('svg');
            df_data.forEach(function(d,i){
                
                // Create 3 text elements with the 'Name' values in df.json
                svg.append('text')
                    .text(function(){return d.Name})
                    .attr('x', i*50)
                    .attr('y', 10);
                
                // Create 3 rectangles with height mapped to the 'Count' values in df.json
                svg.append('rect')
                    .attr('width', 10)
                    .attr('height', function(){ return d.Count * 5})
                    .attr('x', i*50)
                    .attr('y', 20)
                    .attr('fill', 'black');
            });
        });
    })
})(element);

### Creating the D3 visualisation from external files in a Jupyter cell

In [ ]:
from IPython.display import HTML

We will use 2 external code files from the same directory as this notebook: viz.js and viz.css.html. I've included their contents below for reference (leave the code commented out).

In [ ]:
# CONTENTS OF viz.js
# // Define a module, using RequireJS syntax, called 'viz'. 
# // 'viz' is defined as a function with argument 'container' (to append the SVG element to)
# // 'viz' draws a 50 x 50 rectangle

# define('viz', ['d3'], function (d3) {
#     function draw(container) {
#         d3.select(container).append("svg").append('rect').attr('id', 'viz_rect').attr('width', 50).attr('height', 50);
#     }
#     return draw;
# });
# element.append('Loaded 😄 ');

In [ ]:
# CONTENTS OF viz.css.html
# <style>
#     #viz_rect {
#         fill: blue;
#     }
# </style>

Load in viz.css.html and viz.js

In [ ]:
display(HTML(filename="viz.css.html"))
Javascript(filename='viz.js')
# You should see the message 'Loaded 😄' below from viz.js, 
# but you won't see any message from viz.css.html to indicate it's loaded

(Using [RequireJS](https://requirejs.org/) syntax) we use the module 'viz', passing in the argument 'element.get(0)' to generate our D3 graphic.

In [ ]:
Javascript("""
(function(element){
    require(['viz'], function(viz) {
        viz(element.get(0))
    });
})(element);
""")
# the css styling in viz.css.html colours this rectangle blue

### Passing variables across cells between Python and JavaScript

#### Python to JavaScript

In [ ]:
variable1 = "this_could_be_any_python_string"
HTML("<div id='variable1' data-fromPython='" + variable1 + "' ></div>")

In [ ]:
%%javascript
var fromPython = document.getElementById('variable1').getAttribute('data-fromPython');
element.text(fromPython);

This method works for Python objects that can be represented as a string and then re-interpreted back once they've been passed into JavaScript, for example a list.

#### JavaScript to Python

In [ ]:
%%javascript
var variable2 = "this_could_be_any_javascript_string";
// command is a string containing Python code
var command = "fromJavaScript='" + variable2 + "'";
IPython.notebook.kernel.execute(command);

In [ ]:
print(fromJavaScript)

Similarly, this method works for JavaScript objects that can be represented as a string and then re-interpreted back once they've been passed into Python, for example an array.